In [1]:
%matplotlib inline
import os
import sys
sys.path.append('../Google_Refexp_toolbox/google_refexp_py_lib/')
sys.path.append('../Google_Refexp_toolbox/coco/PythonAPI/')
sys.path.append('..')
sys.path.append('../models/')
sys.path.append('../util/')

from refexp import Refexp

import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

from IPython.display import display, clear_output
import ipywidgets

import os

from nltk.tree import Tree
from nltk.draw.tree import TreeView
from nltk.draw.util import CanvasFrame
from nltk.draw import TreeWidget

from tree import Tree as T
from random import shuffle
%reload_ext autoreload
%autoreload 2

In [2]:
import os, h5py, json, argparse

import numpy as np
import cPickle as pickle

import torch
import torch.nn as nn
import torch.optim as optim
import ipywidgets as widgets
widgets.Button(description='Hello')
from model_util import makevar, printVec, get_n_params, get_box_feats, lamm, mbr
from get_model import get_model


In [3]:
# Specify datasets path.
coco_filename       = '../data/instances_train2014.json'
refexp_filename_val = '../data/google_refexp_val_201511_coco_aligned.json'
image_dir  = '../data/refexp_test/'
imdb_file  = '../data/imdb/imdb_val.npy'
parse_root = '../data/refcocog_sentence/val'

#refexp_filename_val = '../data/google_refexp_train_201511_coco_aligned.json'
#image_dir  = '../data/refexp_val/'
#imdb_file  = '../data/imdb/imdb_trn.npy'
#parse_root = '../data/refcocog_sentence/trn'


blacklist_file = '../data/blacklist.txt'
blacklist = set([line.strip() for line in open(blacklist_file)])
val_imdb = np.load(imdb_file)
# Create Refexp instance.
refexp_val = Refexp(refexp_filename_val, coco_filename)

imIds = refexp_val.getImgIds()
im2id = {}
id2im = {}
for img_id in imIds:
    img = refexp_val.loadImgs(img_id)[0]
    img_name = img['file_name']    
    id2im[img_id]   = img_name
    im2id[img_name] = img_id 


loading annotations into memory...
Done (t=9.45s)
creating index...
index created!
Dataset loaded.


In [4]:
args = argparse.ArgumentParser()
args.resume = '../downloaded_models/snapshot.groundnet.model'
args.dump = '../data/stanford_cmn_refcocog_iou05.triplets.pkl'

# args.resume = '../downloaded_models/snapshot.cmn.model'
# args.dump = '../data/stanford_cmn_refcocog_iou05.NOtriplets.pkl'

args.cnn  = '../data/stanford_cmn_refcocog_iou05.box+meta+smax.h5'

CNN  = h5py.File(args.cnn, 'r')
data = pickle.load(open(args.dump))

val     = data['tst'] ## Google's val is our test. They did not release test set, yet.
#val     = data['dev']

vocabs  = data['vocabs']
tst_json= data['tst_json']

vectors, w2i, p2i, n2i, i2w, i2p, i2n = vocabs
Xval_tree, Xval_box, Xval_iou, Yval   = val

indexes = range(len(Xval_tree))

if args.resume != '':
    net = torch.load(args.resume, map_location=lambda storage, locatoin: storage.cuda(0))
    net.debug = False
    net.config['debug'] = False

    config = net.config
    net.eval()
else:
    raise NotImplementedError()

print("="*20)
print("Starting debugging {} model".format(config['model']))
print("model: {} for {} instances for {} parameters".format(config['model'], len(indexes), get_n_params(net)))

Starting debugging groundnet model
model: groundnet for 9536 instances for 28400575 parameters


In [5]:
def drawTree(sent,name):
    cf = CanvasFrame()
    t = Tree.fromstring(sent)
    tc = TreeWidget(cf.canvas(),t)
    tc['node_font'] = 'ubuntu 18 bold'
    tc['leaf_font'] = 'ubuntu 16 bold'
#     tc['node_color'] = '#007996'
#     tc['leaf_color'] = '#668D3C'
#     tc['line_color'] = '#816C5B'
    
    tc['node_color'] = '#636363'
    tc['leaf_color'] = '#000000'
    tc['line_color'] = '#000000'
    
    cf.add_widget(tc,0,0) # (10,10) offsets
    cf.print_to_file('{}.ps'.format(name))
    os.system('convert {}.ps {}'.format(name,name))
    cf.destroy()

def drawTrees(fname):
    l = [line.strip().replace("@","") for line in open(fname)]
    tree = T.from_sexpr(l[0])
    drawTree(tree.children[0].children[1].children[1].getRaw(),'tree.png')
    #drawTree(l[0],'tree.png')
    t = T.from_sexpr(l[0])
    t.preClean()                                                                                                                                  
    t.adopt()                                                                                                                                     
    t.adopt()                                                                                                                                     
    t.preTriplet(blacklist)                                                                                                                       
    to3 = t.children[0] if len(t.children) == 1 else t                                                                                            
    d,ct_raw = to3.findTriplet(blacklist)                                                                                          
    drawTree(ct_raw, 'ctree.png')
    
def getSentences(instance, refexp, im2id):
    im_name = instance['im_path'].split('/')[-1]
    img_id = im2id[im_name]
    regions = _load_refexp_regions(refexp, img_id)

    s_list = []
    for i,region in enumerate(regions):
        bbox,sentences,ann_id = region 
        if sentences != []:
            s_list += sentences
    return s_list

def plotInstance(instance, refexp, image_dir, im2id, gid = [], pid = 0,fname = 'prediction.png', add_text = True):
    
    im_name = instance['im_path'].split('/')[-1]
    img_id = im2id[im_name]
    img = refexp.loadImgs(img_id)[0]
    assert(img['file_name'] == im_name)
    im_path = os.path.join(image_dir, img['file_name'])
    regions = _load_refexp_regions(refexp, img_id)
        
    fig = plt.figure()

    
    ax = plt.gca()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
              
    I = io.imread(im_path)
    imshape = I.shape
    ax.imshow(I)

            
    gold = False
    idx = 0
    drawn = set()
    lwidth = 2
    for i,region in enumerate(regions):
        bbox,sentences,ann_id = region
        pcolor = '#FF9642'
        if i == pid:
            pcolor = '#C0362C'
        if i in set(gid):
            pcolor = '#007996'
        if i == pid and i in set(gid):
            pcolor = '#668D3C'
            
        box_plot = Rectangle((bbox[0], bbox[1]), bbox[2], bbox[3], fill=False, edgecolor=pcolor, linewidth=lwidth)
        ax.add_patch(box_plot)
        if add_text:
            ax.text(bbox[0]-25, bbox[1]-15, '{}'.format(i),
                    bbox=dict(facecolor='#ad4141',alpha = 0.5),fontsize = 18, color = 'white')
    fig.tight_layout()
    fig.savefig(fname)
    plt.close()

    return imshape

def plotDataset(imdb, refexp, image_dir, im2id):

    imgIds = refexp.getImgIds()
    for i,instance in enumerate(imdb):
        plotInstance(instance,refexp,image_dir,im2id)
        fname = os.path.join(parse_root,str(i)+'_0.np')
        drawTrees(fname)
        if i == 2:
            return

def _load_refexp_regions(refexp, img_id):
    ann_ids_gref = refexp.getAnnIds(img_id)
    anns_gref = refexp.loadAnns(ann_ids_gref)
    ann_ids_coco = refexp.coco.getAnnIds(img_id)
    anns_coco = refexp.coco.loadAnns(ann_ids_coco)
    
    # Align refexp annotations to those in coco
    anns_gref_map = {ann_ids_gref[n]: anns_gref[n] for n in range(len(ann_ids_gref))}
    for n in range(len(ann_ids_coco)):
        if ann_ids_coco[n] in anns_gref_map:
            anns_coco[n]['refexp_ids'] = anns_gref_map[ann_ids_coco[n]]['refexp_ids']
        else:
            anns_coco[n]['refexp_ids'] = []
    
    regions = [[ann['bbox'],
                _load_refexp_sentences(refexp, ann['refexp_ids']),
                ann['id']] for ann in anns_coco]
    return regions

def _load_refexp_sentences(refexp, refexp_ids):
    exps = [e['raw'] for e in refexp.loadRefexps(refexp_ids)]
    return exps

In [6]:
def predict(tree, box_rep, header, gold, verbose = False):
    if "groundnet" in config['model']:
        prediction = net(tree, box_rep, tree, decorate = True)
    else:
        prediction = net([w2i.get(n.label,0) for n in tree.leaves()], box_rep, tree)

    _,pred = torch.max(prediction.data,1)
    
    if verbose:
        print
        print "="*15,header,"="*15
        print "fname:",tree.fname
        print "Gold: {} Prediction: {}".format(gold,pred[0][0])
        print "_"*50
        print "Inner Nodes:"
        for n in tree.nonterms():
            if n.label != "and":
                context = ""
                attn = printVec(n._attn.t(),width = 2)
                if n.label == "loc":
                    label = n.children[0].label
                else:
                    label = n.label                
                for tok,prob in zip(label.split('_'),attn.split(" ")):
                    context += tok+"("+prob+") "
            else:
                context = "and"
            #print printVec(n._expr,logprob = True,width = 2),"\t",context
    return pred[0][0]
    
def run_instance(ii, cg = None):
    tree = Xval_tree[ii]
    box_feats, spat_feats = get_box_feats(Xval_box[ii], CNN, volatile = True)


    if config['box_usage'] == 0:
        box_rep = torch.cat((box_feats,spat_feats),1)
    elif config['box_usage'] == 1:
        box_rep = box_feats
    elif config['box_usage'] == 2:
        box_rep = spat_feats
    else:
        raise NotImplementedError()
    trn_idx = int(tree.fname.split('/')[-1].split('_')[0])
    sen_idx = int(tree.fname.split('/')[-1].split('_')[1].split('.')[0])
    if cg == None:
        prediction = predict(tree, box_rep, "Parser Output", Yval[ii])
    else:
        prediction = predict(cg, box_rep, "Parser Output", Yval[ii])

    return trn_idx, sen_idx ,Yval[ii] , prediction, os.path.join(parse_root , tree.fname.split('/')[-1].replace(".np",".txt"))

In [7]:
label_nl = ipywidgets.Label(value="Referring Expression")
label_cg = ipywidgets.Label(value="TComputation Graph")

ibutton_next = ipywidgets.Button(description="Next")
ibutton_prev = ipywidgets.Button(description="Previous")

islider = ipywidgets.IntSlider(description='Instance Number')

input_nl = ipywidgets.Text(width='40%',description="")
input_cg = ipywidgets.Text(width='40%',description="")

ibutton_nl = ipywidgets.Button(description="Locate NL Input")
ibutton_cg = ipywidgets.Button(description="Locate CG Input")

def on_nexti(b):
    islider.value += 1
    on_button_clicked()
    
def on_previousi(b):
    islider.value -= 1
    on_button_clicked()

def handle_slider_change(change):
    on_button_clicked()

def on_button_clicked():
    idx = islider.value
    instance_idx, sid, gid, pid, fname = run_instance(idx)
    
    instance = val_imdb[instance_idx]
    shape = plotInstance(instance,refexp_val,image_dir,im2id, gid = gid, pid = pid)
    _     = plotInstance(instance,refexp_val,image_dir,im2id, gid = gid, pid = pid,
                         add_text = False,fname = 'prediction.noboxid.png')

    fname = os.path.join(parse_root,str(instance_idx)+'_'+str(sid)+'.np')
    drawTrees(fname)
    tf = open("tree.png", "rb")
    tree = tf.read()    
    tf = open("prediction.png", "rb")
    anns = tf.read()
    tf = open("ctree.png", "rb")
    ctree = tf.read()  
    input_nl.value = ""
    input_cg.value = ""
    
    c = 0.75
    tc = 0.5
    w = 300
    if not on_button_clicked.image_on:
        TREE  = ipywidgets.Image(value=tree,format='png',width=w,height=w)
        ANNS  = ipywidgets.Image(value=anns,format='png',width=int(shape[1]*c),height=int(shape[1]*c))
        CTREE = ipywidgets.Image(value=ctree,format='png',width=w,height=w)
        
        CGTREE = ipywidgets.Image(value=ctree,format='png',width=w*tc,height=w*tc)
        CGANNS = ipywidgets.Image(value=anns, format='png',width=w*tc,height=w*tc)

        on_button_clicked.image_on = True
        on_button_clicked.TREE  = TREE
        on_button_clicked.ANNS  = ANNS
        on_button_clicked.CTREE = CTREE
        
        on_button_clicked.CGTREE= CGTREE
        on_button_clicked.CGANNS= CGANNS
    else:
        on_button_clicked.TREE.value  = tree
        on_button_clicked.ANNS.value  = anns
        on_button_clicked.CTREE.value = ctree
        
        on_button_clicked.CGTREE.value= ctree
        on_button_clicked.CGANNS.value= anns
        

parser_root='../parser/'
parser='stanford-parser-full-2016-10-31/stanford-parser.jar'
parser_model='stanford-parser-full-2016-10-31/stanford-parser-3.7.0-models.jar'


def on_user_input(cg):
    if cg == None:
        return
    cg.fname = "stdin"
    idx = islider.value
    instance_idx, sid, gid, pid, fname = run_instance(idx,cg = cg)
    
    instance = val_imdb[instance_idx]
    shape = plotInstance(instance,refexp_val,image_dir,im2id, gid = gid, pid = pid, fname = 'prediction_user.png')    

    drawTree(cg.getRaw(),"cgtree.png")
    cg_file = open("cgtree.png", "rb")
    cg_tree = cg_file.read()    

    cg_file = open("prediction_user.png", "rb")
    cg_anns = cg_file.read()    

    on_button_clicked.CGTREE.value= cg_tree
    on_button_clicked.CGANNS.value= cg_anns
    
def parse_nl(b):
    f = open(os.path.join(parser_root,'example.txt'),'w')
    f.write(input_nl.value+'\n')
    f.close()
    
    cp = '{}:{}:{}'.format(parser_root,parser_root+parser,parser_root+parser_model)
    cmd='java -cp {} ConstituencyParse -tokpath {}.tok -parentpath {}.par -tokenize -inputpath {}.txt -bparsepath {}.bp -nparsepath {}.np'.format(cp,parser_root+'example',parser_root+'example',parser_root+'example',parser_root+'example',parser_root+'example')
    os.system(cmd)
    try:
        l = [line.strip() for line in open(os.path.join(parser_root,'example.np'))][0]
        t = T.from_sexpr(l)
        t.preClean()
        t.adopt()
        t.adopt()
        t.preTriplet(blacklist)
        to3 = t.children[0] if len(t.children) == 1 else t
        d,ct_raw = to3.findTriplet(blacklist, mode = 1)
        ct = T.from_sexpr(ct_raw)
        ct.original_text = input_nl.value
        print ct.getRaw()
    except:
        ct = None
        print "Cannot create computation graph with given input"
        pass
    input_cg.value = ""
    on_user_input(ct)
    clear_output(wait=True)
    display(ihbox,imbox,tbox,inputs)
    
def parse_cg(b):
    os.system('pwd')
    try:
        t = T.from_sexpr(input_cg.value.split('|')[0])
        t.original_text = input_cg.value.split('|')[1]
        print t.getRaw()
    except:
        t = None
        print "Cannot create computation graph with given input"
        pass
    input_nl.value = ""
    on_user_input(t)
    clear_output()
    display(ihbox,imbox,tbox,inputs)

    

ibutton_nl.on_click(parse_nl)
ibutton_cg.on_click(parse_cg)

on_button_clicked.image_on = False
islider.max = len(val_imdb)-1
islider.value = 0
islider.observe(handle_slider_change, names='value')

ibutton_next.on_click(on_nexti)
ibutton_prev.on_click(on_previousi)

on_button_clicked()
NL   = ipywidgets.HBox([label_nl,input_nl,ibutton_nl])
CG   = ipywidgets.HBox([label_cg,input_cg,ibutton_cg])
FIGS = ipywidgets.HBox([on_button_clicked.CGTREE])

inputs = ipywidgets.VBox([NL,CG,FIGS])
ihbox  = ipywidgets.HBox([ibutton_prev, ibutton_next, islider])
imbox  = ipywidgets.HBox([on_button_clicked.ANNS])
tbox   = ipywidgets.HBox([on_button_clicked.TREE,on_button_clicked.CTREE]) 

display(ihbox,imbox,tbox,inputs)


/home/vcirik/bin/anaconda2/envs/pytorch0.1.12/lib/python2.7/site-packages/torch/backends/cudnn/__init__.py:40: UserWarning: PyTorch was compiled without cuDNN support. To use cuDNN, rebuild PyTorch making sure the library is visible to the build system.
  "PyTorch was compiled without cuDNN support. To use cuDNN, rebuild "


HBox(children=(Button(description=u'Previous', style=ButtonStyle()), Button(description=u'Next', style=ButtonStyle()), IntSlider(value=0, description=u'Instance Number', max=4649)))

HBox(children=(Image(value='\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\xb0\x00\x00\x01 \x08\x06\x00\x00\x00\xd5(&i\x00\x00\x00\x04sBIT\x08\x08\x08\x08|\x08d\x88\x00\x00\x00\tpHYs\x00\x00\x0b\x12\x00\x00\x0b\x12\x01\xd2\xdd~\xfc\x00\x00\x009tEXtSoftware\x00matplotlib version 2.2.0, http://matplotlib.org/\x16\xf9\xf2V\x00\x00 \x00IDATx\x9c\xec\xbdy\xb0l\xd9U\xde\xf9[k\xefsr\xbcy\xa77\x8f5\xaa\xaa$\x95\x90\x84TX\x08$\xc0\x88!\xcch\xbb\xd5\xb8e\x9ap\xd3\xc6\xc6mw\xd3\xd0\x1da\x871\x98\xc0a\xa60\xd0\x98\x8en\xcbm\x0c22\x93\x0cX#\x83$JC\xa9\xa4\x92T\xa5\x9a\x87W\xf3{\xf5\xe6w\xe7{\xf3f\xe6\xd9{\xf5\x1f{\x9f\xcc\xbc\xaf\xc0vw\xd0@\xdd\xce\xefE\xc6\xbb7\xf3\x0c\xfb\x9c\x93w\xaf\xbd\xd6\xfa\xd6\xb7\xc4\xcc\x8c\x19f\x98a\x86\x19fx\x85A\xff\xbc\x070\xc3\x0c3\xcc0\xc3\x0c\xffo03`3\xcc0\xc3\x0c3\xbc"13`3\xcc0\xc3\x0c3\xbc"13`3\xcc0\xc3\x0c3\xbc"13`3\xcc0\xc3\x0c3\xbc"13`3\xcc0\xc3\x0c3\xbc"13`3\xcc0\xc3\x0c3\xbc"13`3\xcc0\xc3\x0c3\xbc"13`3\xcc0\xc3\x0c3\xbc"13`3\xcc0\xc3\x0c3\xbc"13`3\xcc0\xc3\x0c3\xbc"13`3\xcc0\xc3\x0c3\xbc"13`3\xcc0\xc3\x0c3\xbc"\xe1\xff\xbc\x070\xc3\xffw(;\xf34\x17\x8e\xe4\xdf\x04\xb0?\xe1\'\x99\xda\xeb\xfa\xe6\x042\xf5\xbe\xbcl\x0f\xbbn\xab\xeb\xcfU\xff.y/\x1b\xefm{Oo\x82\x08`F\xfa\xe1?\x83=\xa7\xb8\xfe|\x7f\xd2\xe7SW.\xc2\x9eF\x0c\xe3\x8f\x04\xc4&?\x8f\x91\xf7\xffcN\xf5\x9f\xbc#b`\x7f\xccq^\xf6\xfb\xde\xfb\xf5\x9f\xde\xfe?\x87?i\xfb\xeb\xc78}\x06\xc9\xefN\xc6\x9fFe/\xbf\xbc\x97\xdf\xce\xc9^\x92\xeem\x0c\x01D\x11\x85\xfe\x95\xb3,t;lmo2\xac\x86\xa8\x16i\xfb\x1803D\x04T\xc7\xe7D\x04Wx\xb08>\xc9\xb0\xbf\x8b\xaa\xa0\xf9\x19\xc8x,\xd3\xdff!b\x14\xe2\xd2\xe1\xf2\xf1\xfa\xc3!\xc3P\xfd\x17\xdc\xb7\x19^i\x90Y;\x95\xfd\x8b\xde\xf1\xdb\xb8\xeb\xfb\xdfM\xfd\x84\xebI{\xfa\x7f\xb1\x88\x89\xa2\xd7\xcd\xe5LM\xf0i\x8aP\xcc\x0cSCU\xf1\xe6\x88\xe3\x89#\x02`QP\xcd\xdb\tD\x01\x8f!Q\x11\x0fV\x01\xce\xc0\x14#\x8c\xb7\x15\x11\x04G\x8c\x11u\x80\xe5\xc0\x80\xa61\xe8u\xe6Q\x11\x02\x15N\x14S\xc1\xc2\xe4\xfaD\r\x11\x88q2\x1d\xa7\xeb\x15\xc4@\xf2\x85\x9aE\x04\x9f&O\t\xc4hi\x0cV\x91\xe6\xd34\xa1\x86*]\xaf\t\x10\xd3X\x83\xc5t.\xc9S\xa6\x19\xa2J\xcc\xdbF\xa9\x90\xe8\xd2y$"\xe2\x10#]_\x9aY1\x02Bz?\x88\xe1\xd4\x88\xf9\xba5&\x9b\x171\xa2\x80\xc3\xd29L\xd1<IG\x0c\x8f\xa6g\xa0\x93g\x95\xb6\xcb\xf7.\xc4\xf1\xfd\xaa\\\xc4[\x91\xb7\x8b{\xb6C\x1d\xb1\n\x88S\x886~\xae\x8a\x80\n1\xa4\xf1\x9a\x1a\x12\x85(\xf9\xb8\xf9;\x13QR0\'\x12\xad\xc2\xa2#\xaa\xa7\xd5\x1c\xf1\xe4\xff\xfe\x03|\xc3;\xde\xc2\x1f|\xec\xc3\xcc\xf5\x16\xb8\xbc\xbeK\x15+\xc2\xce\x06\xa3\xc1\x0ee\xb3\x834\x1a \x05jP\xf6:t\xe6{\xd8p\x9ba\xd1\xa2[\x1ag\xbf\xf4\x08\xed\xb2\xa0\x14\xc1\xa9\x82iz\x8eQ\xb2\xc5L\xcf\xa3\x11\x85\xf9^\x87\x93\xbdE\xae\xad\xad\x13\x9c\xe1\xc5s\xef3O\xb1\xbe\xdb\xff\x7f\xf6\xc73\xc3+\x023\x0fl_C\x88\xd1(\xb4\xa0\xb2\x08\xa4\t\xd8#X\xed\xe5HA\x8c\x11Q\xd2\x840\xb6d\x8aHL\xff\xe7\xd5\xb6sy"\x0e\x11\\6\x00!`J6\x04\xc9\x18$\xc3\x98\x8c\xa2\xbad\x8f\xcc\x0c\xf1\x02\xe2\x111,\xea\xd8pA2,*\xa4\xc9\xa9\x1e\x9bF\xd4$\x1b\x9bd\xe0D\x04\xf1 \x95C\x9c\xe4!\xd7\x93\x18\xd9\x08\x08\xaai\x92\x8df\x93k\xcbC\x8cfD\x03\xef\x01\x8c\x10\x92\xe1!\x82we2bx\x8c\x80\xf3S+|Ub\x8cxM\xc6[\xc4PQ\x0c%`\xb8B\xd2\x18\xf1\xa0\xa4\xcf\xcd\x8f\x8d\xb4\xaa&\x83\x11\r\xd1\xf4>\x9a<\x9fh\x82\xe6\xfb4\xd2t\xdf\x1dB!\x06\xe21\x0b\x88\xa5\xfd\xc5\x0c5#\x02"i\x11\xa0\x9a\xeek\x8c1\x19J\x11\xd49,\x7f\xeeE\x937\x18\xb2A\xcd\xb7\xd8,\xed\xa3\xde!\x16\xd3\xb8-\x1b\xc6\xbc\x91\x90\x86m8B6\xf4\xde\xc9\xf8\xb9b\x96\xc7\x97\xbcj\xd1!6Z\xe1\xb3\xef\xfeQ~\xe1\'~\x88_\xfc\xf9\x7f\xcb\xda\xfa:}k#\x18\xc3\xcdUJ\xef\x90v\x0f)J }\xb7:K\x8b\xf8R\taD\xbb\xdd\xe2\xe2C\x0f\xb2-\x8e\xf9N\x9b"N\x161\xe2\x0c1G "\x80\x8b\xb0\xd0j\xb0P\xb6\x88\x04\x9cEp`U\x00\xe7\xb2\xd7?\xc3~\xc4\xcc\x80\xeds\xa4\xc9\xca\x10\xd2\xea\xd9)\x88\x96X\xac0\x0b(\x8a\x13@\x04q\x91hi\x02\x03PW\xaf\xec=BL\xd1\xb0\xda\x9d3\x05\x0b\xe0\x0c\xa7\x8e\x18\xc0\x89`*H\x90\xbcI\x9e\xcc\xcdp\x9a\xbd\x8a\x18\x88\xaa8\xe7\xb0\x18\xd3J\xda<J \xa2\x88N\x8c\x11\xe3\x95}20\xa26\xf6\x0c\xbdK\x13\xbfF\xc3\x89\x03u\x0c\xab\x01\xce+\x98#\x9a\xa1\xd9KJ\xfb\xe7a[2\xb6E\x99\xbf\xfa\x

HBox(children=(Image(value='\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01}\x00\x00\x01\x0c\x08\x04\x00\x00\x00\x82\x0e\xae\xbe\x00\x00\t)iCCPicc\x00\x00x\xda\x95\x91gP\x94\x87\x16\x86\xcf\xf7}\xdb\x0bm\x97\xa5\xc3\xd2\x9bT)\x0bHYz\x95^E\x05\x96\xdeY\x96"bC\xc4\x08D\x14\x11i\x8a \xa2\x80\x82Q)\x12+\xa2X\x08\n\x8aX\xd0,\x12\x04\x94\x18\x8c"*(\xf7G\xeeL\x9c{\'?\xf2\xfcz\xe6\x9dw\xce93\x07\x80"\x06\x00\x80\x8a\x01\xa4\xa4\n\xf8~.\xf6\xec\x90\xd006|G$/3\x9d\xeb\xe3\xe3\t\xff\xc8\xc7Q@\x00\x00\x1e\xac\x82\x7f\x0f%:&\x93\x07\x00\xcb\x00\x90\xcfK\xe7\x0b\x00\x90\\\x00\xd0\xca\x11\xa4\x0b\x00\x90\xa3\x00\xc0\x8cJJ\x17\x00 \xe7\x01\x80\xc9\x0f\t\r\x03@n\x01\x003\xee/\x1f\x07\x00f\xd4_>\x0f\x00L~\x80\x9f\x03\x00\x8a\x03 \xd1\xe2\xbe\xf3\xa8\xef\xfc\xbf{\x01\x00T\xb8|ABlL.\xdb?-V\x90\x13\xc9\x8fag\xfa\xb9\xd8\xb3\xdd\x1c\x1c\xd8>\xfc\xb4\xd8\x84\xe4\x98\xef\x0e\xfeW\xe5\x7f@\x10\x93+\x00\x00pHK\xdf\xc4O\x88\x8b\x17\xb0\xffo\xa8\xb1\xa1\x91\x11\xfc\xfd\x8b\xf7\xbe\x80\x00\x00\xc2\x1a\xfc\xdf\xff\x00\xc0w\xbd\xb4F\x00\xce\x02\x00\xb6\xef\xef,\xaa\x1a\xa0{\x17\x80\xf4\xd3\xbf3\xb5\xa3\x00\xa2\x85\x00]\xf7xY\xfc\xec\xbf2\x1c\x00\x00\x1e( \nL\x90\x01EP\x05-\xd0\x03c0\x07+\xb0\x03\'p\x07o\x08\x80P\xd8\x00<\x88\x87\x14\xe0C\x0e\xe4\xc3\x0e(\x82\x12\xd8\x07\x07\xa1\x06\xea\xa1\tZ\xa0\x1d\xceB7\\\x84kp\x13\xee\xc2}\x18\x85g \x84)x\x03\xf3\xf0\x11\x96\x10\x04!"t\x84\x81\xc8 J\x88:\xa2\x8b\x18#\x1c\xc4\x06qB<\x11?$\x14\x89@\xe2\x90T$\x0b\xc9Gv"%H9R\x834 -\xc8O\xc8\x05\xe4\x1ar\x1b\x19F\x9e \x13\xc8,\xf2\'\xf2\x05\xc5P\x1a\xcaD\x15P\r\xd4\x00\xe5\xa0\\\xd4\x03\r@\xd7\xa3qh\x06\x9a\x87\x16\xa2{\xd1*\xb4\x11=\x85v\xa1\xd7\xd0\xbb\xe8(*D\xdf\xa0\x0b\x18`T\x8c\x85)cz\x18\x07s\xc0\xbc\xb10,\x16\xe3c[\xb1b\xac\x12k\xc4\xda\xb1^l\x00{\x80\t\xb19\xec3\x8e\x80c\xe0\xd88=\x9c\x15\xce\x15\x17\x88\xe3\xe12p[q\xa5\xb8\x1a\xdcI\\\x17\xae\x1f\xf7\x007\x81\x9b\xc7}\xc3\xd3\xf1\xf2x]\xbc%\xde\r\x1f\x82\x8f\xc3\xe7\xe0\x8b\xf0\x95\xf8f|\'\xfe\x06~\x14?\x85\xffH \x10X\x04M\x829\xc1\x95\x10JH$l&\x94\x12\x0e\x13:\x08W\t\xc3\x84I\xc2\x02\x91H\x94!\xea\x12\xad\x89\xde\xc4H\xa2\x80XD\xac&\x9e"^!\x8e\x10\xa7\x88\x9fHT\x92\x12\xc9\x98\xe4L\n#\xa5\x92\nH\x95\xa4V\xd2e\xd2\x08i\x9a\xb4D\x16#\xab\x93-\xc9\xde\xe4h\xf2&r\x19\xb9\x89\xdcK\xbeG\x9e"/Q\xc4)\x9a\x14kJ\x00%\x91\xb2\x83REi\xa7\xdc\xa0\x8cS\xdeS\xa9T\x15\xaa\x05\xd5\x97\x9a@\xddN\xad\xa2\x9e\xa1\xde\xa2NP?\xd3$h:4\x07Z8-\x8b\xb6\x97v\x82v\x95\xf6\x84\xf6\x9eN\xa7k\xd0\xed\xe8at\x01}/\xbd\x85~\x9d\xfe\x82\xfeI\x84!\xa2/\xe2&\x12-\xb2M\xa4V\xa4KdD\xe4\xad(YT]\x94+\xbaA4O\xb4R\xf4\x9c\xe8=\xd191\xb2\x98\x86\x98\x83X\xa4\xd8V\xb1Z\xb1\x0bbcb\x0b\xe2\x0cq#qo\xf1\x14\xf1R\xf1V\xf1\xdb\xe23\x12D\t\r\t\'\x89h\x89B\x89c\x12\xd7%&\x19\x18C\x95\xe1\xc0\xe01v2\x9a\x187\x18SL\x02S\x93\xe9\xc6Ld\x960O3\x87\x98\xf3\x92\x12\x92&\x92A\x92\xb9\x92\xb5\x92\x97$\x85,\x8c\xa5\xc1rc%\xb3\xcaXgY\x8fX_\xa4\x14\xa4\xb8R1R{\xa4\xda\xa5F\xa4\x16\xa5\xe5\xa4\xed\xa4c\xa4\x8b\xa5;\xa4G\xa5\xbf\xc8\xb0e\x9cd\x92d\xf6\xcbt\xcb<\x97\xc5\xc9\xea\xc8\xfa\xca\xe6\xc8\x1e\x91\xbd!;\'\xc7\x94\xb3\x92\xe3\xc9\x15\xcb\x9d\x95{*\x8f\xca\xeb\xc8\xfb\xc9o\x96?&?(\xbf\xa0\xa0\xa8\xe0\xa2\x90\xaeP\xadp]aN\x91\xa5h\xa7\x98\xa8X\xa1xYqV\x89\xa1d\xa3\x94\xa0T\xa1tE\xe95[\x92\xcde\'\xb3\xab\xd8\xfd\xecyeyeW\xe5,\xe5\x06\xe5!\xe5%\x15M\x95@\x95\x02\x95\x0e\x95\xe7\xaa\x14U\x8ej\xacj\x85j\x9f\xea\xbc\x9a\x92\x9a\x97Z\xbeZ\x9b\xdaSu\xb2:G=^\xfd\x90\xfa\x80\xfa\xa2\x86\xa6F\xb0\xc6n\x8dn\x8d\x19MiM7\xcd<\xcd6\xcdq-\xba\x96\xadV\x86V\xa3\xd6Cm\x826G;I\xfb\xb0\xf6}\x1dT\xc7T\'^\xa7V\xe7\x9e.\xaak\xa6\x9b\xa0{Xwx\x15~\x95\xc5\xaa\xd4U\x8d\xab\xc6\xf4hz\\\xbdl\xbd6\xbd\t}\x96\xbe\xa7~\x81~\xb7\xfe[\x035\x830\x83\xfd\x06\x03\x06\xdf\x0cM\r\x93\r\x9b\x0c\x9f\x19I\x18\xb9\x1b\x15\x18\xf5\x1a\xfdi\xacc\xcc3\xae5~\xb8\x9a\xbe\xday\xf5\xb6\xd5=\xab\xdf\x99\xe8\x9a\xc4\x98\x1c1yl\xca0\xf52\xddm\xdag\xfa\xd5\xcc\xdc\x8co\xd6n6k\xaef\x1ea^g>\xc6ar|8\xa5\x9c[\x16x\x0b{\x8bm\x16\x17->[\x9aY\n,\xcfZ\xfea\xa5g\x95d\xd5j5\xb3FsM\xcc\x9a\xa65\x93\xd6*\xd6\x91\xd6\r\xd6B\x1b\xb6M\x84\xcdQ

VBox(children=(HBox(children=(Label(value=u'Referring Expression'), Text(value=u''), Button(description=u'Locate NL Input', style=ButtonStyle()))), HBox(children=(Label(value=u'TComputation Graph'), Text(value=u''), Button(description=u'Locate CG Input', style=ButtonStyle()))), HBox(children=(Image(value='\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01}\x00\x00\x01\x0c\x08\x04\x00\x00\x00\x82\x0e\xae\xbe\x00\x00\t)iCCPicc\x00\x00x\xda\x95\x91gP\x94\x87\x16\x86\xcf\xf7}\xdb\x0bm\x97\xa5\xc3\xd2\x9bT)\x0bHYz\x95^E\x05\x96\xdeY\x96"bC\xc4\x08D\x14\x11i\x8a \xa2\x80\x82Q)\x12+\xa2X\x08\n\x8aX\xd0,\x12\x04\x94\x18\x8c"*(\xf7G\xeeL\x9c{\'?\xf2\xfcz\xe6\x9dw\xce93\x07\x80"\x06\x00\x80\x8a\x01\xa4\xa4\n\xf8~.\xf6\xec\x90\xd006|G$/3\x9d\xeb\xe3\xe3\t\xff\xc8\xc7Q@\x00\x00\x1e\xac\x82\x7f\x0f%:&\x93\x07\x00\xcb\x00\x90\xcfK\xe7\x0b\x00\x90\\\x00\xd0\xca\x11\xa4\x0b\x00\x90\xa3\x00\xc0\x8cJJ\x17\x00 \xe7\x01\x80\xc9\x0f\t\r\x03@n\x01\x003\xee/\x1f\x07\x00f\xd4_>\x0f\x00L~\x80\x9f\x03\x00\x8a\x03 \xd1\xe2\xbe\xf3\xa8\xef\xfc\xbf{\x01\x00T\xb8|ABlL.\xdb?-V\x90\x13\xc9\x8fag\xfa\xb9\xd8\xb3\xdd\x1c\x1c\xd8>\xfc\xb4\xd8\x84\xe4\x98\xef\x0e\xfeW\xe5\x7f@\x10\x93+\x00\x00pHK\xdf\xc4O\x88\x8b\x17\xb0\xffo\xa8\xb1\xa1\x91\x11\xfc\xfd\x8b\xf7\xbe\x80\x00\x00\xc2\x1a\xfc\xdf\xff\x00\xc0w\xbd\xb4F\x00\xce\x02\x00\xb6\xef\xef,\xaa\x1a\xa0{\x17\x80\xf4\xd3\xbf3\xb5\xa3\x00\xa2\x85\x00]\xf7xY\xfc\xec\xbf2\x1c\x00\x00\x1e( \nL\x90\x01EP\x05-\xd0\x03c0\x07+\xb0\x03\'p\x07o\x08\x80P\xd8\x00<\x88\x87\x14\xe0C\x0e\xe4\xc3\x0e(\x82\x12\xd8\x07\x07\xa1\x06\xea\xa1\tZ\xa0\x1d\xceB7\\\x84kp\x13\xee\xc2}\x18\x85g \x84)x\x03\xf3\xf0\x11\x96\x10\x04!"t\x84\x81\xc8 J\x88:\xa2\x8b\x18#\x1c\xc4\x06qB<\x11?$\x14\x89@\xe2\x90T$\x0b\xc9Gv"%H9R\x834 -\xc8O\xc8\x05\xe4\x1ar\x1b\x19F\x9e \x13\xc8,\xf2\'\xf2\x05\xc5P\x1a\xcaD\x15P\r\xd4\x00\xe5\xa0\\\xd4\x03\r@\xd7\xa3qh\x06\x9a\x87\x16\xa2{\xd1*\xb4\x11=\x85v\xa1\xd7\xd0\xbb\xe8(*D\xdf\xa0\x0b\x18`T\x8c\x85)cz\x18\x07s\xc0\xbc\xb10,\x16\xe3c[\xb1b\xac\x12k\xc4\xda\xb1^l\x00{\x80\t\xb19\xec3\x8e\x80c\xe0\xd88=\x9c\x15\xce\x15\x17\x88\xe3\xe12p[q\xa5\xb8\x1a\xdcI\\\x17\xae\x1f\xf7\x007\x81\x9b\xc7}\xc3\xd3\xf1\xf2x]\xbc%\xde\r\x1f\x82\x8f\xc3\xe7\xe0\x8b\xf0\x95\xf8f|\'\xfe\x06~\x14?\x85\xffH \x10X\x04M\x829\xc1\x95\x10JH$l&\x94\x12\x0e\x13:\x08W\t\xc3\x84I\xc2\x02\x91H\x94!\xea\x12\xad\x89\xde\xc4H\xa2\x80XD\xac&\x9e"^!\x8e\x10\xa7\x88\x9fHT\x92\x12\xc9\x98\xe4L\n#\xa5\x92\nH\x95\xa4V\xd2e\xd2\x08i\x9a\xb4D\x16#\xab\x93-\xc9\xde\xe4h\xf2&r\x19\xb9\x89\xdcK\xbeG\x9e"/Q\xc4)\x9a\x14kJ\x00%\x91\xb2\x83REi\xa7\xdc\xa0\x8cS\xdeS\xa9T\x15\xaa\x05\xd5\x97\x9a@\xddN\xad\xa2\x9e\xa1\xde\xa2NP?\xd3$h:4\x07Z8-\x8b\xb6\x97v\x82v\x95\xf6\x84\xf6\x9eN\xa7k\xd0\xed\xe8at\x01}/\xbd\x85~\x9d\xfe\x82\xfeI\x84!\xa2/\xe2&\x12-\xb2M\xa4V\xa4KdD\xe4\xad(YT]\x94+\xbaA4O\xb4R\xf4\x9c\xe8=\xd191\xb2\x98\x86\x98\x83X\xa4\xd8V\xb1Z\xb1\x0bbcb\x0b\xe2\x0cq#qo\xf1\x14\xf1R\xf1V\xf1\xdb\xe23\x12D\t\r\t\'\x89h\x89B\x89c\x12\xd7%&\x19\x18C\x95\xe1\xc0\xe01v2\x9a\x187\x18SL\x02S\x93\xe9\xc6Ld\x960O3\x87\x98\xf3\x92\x12\x92&\x92A\x92\xb9\x92\xb5\x92\x97$\x85,\x8c\xa5\xc1rc%\xb3\xcaXgY\x8fX_\xa4\x14\xa4\xb8R1R{\xa4\xda\xa5F\xa4\x16\xa5\xe5\xa4\xed\xa4c\xa4\x8b\xa5;\xa4G\xa5\xbf\xc8\xb0e\x9cd\x92d\xf6\xcbt\xcb<\x97\xc5\xc9\xea\xc8\xfa\xca\xe6\xc8\x1e\x91\xbd!;\'\xc7\x94\xb3\x92\xe3\xc9\x15\xcb\x9d\x95{*\x8f\xca\xeb\xc8\xfb\xc9o\x96?&?(\xbf\xa0\xa0\xa8\xe0\xa2\x90\xaeP\xadp]aN\x91\xa5h\xa7\x98\xa8X\xa1xYqV\x89\xa1d\xa3\x94\xa0T\xa1tE\xe95[\x92\xcde\'\xb3\xab\xd8\xfd\xecyeyeW\xe5,\xe5\x06\xe5!\xe5%\x15M\x95@\x95\x02\x95\x0e\x95\xe7\xaa\x14U\x8ej\xacj\x85j\x9f\xea\xbc\x9a\x92\x9a\x97Z\xbeZ\x9b\xdaSu\xb2:G=^\xfd\x90\xfa\x80\xfa\xa2\x86\xa6F\xb0\xc6n\x8dn\x8d\x19MiM7\xcd<\xcd6\xcdq-\xba\x96\xadV\x86V\xa3\xd6Cm\x826G;I\xfb\xb0\xf6}\x1dT\xc7T\'^\xa7V\xe7\x9e.\xaak\xa6\x9b\xa0{Xwx\x15~\x95\xc5\xaa\xd4U\x8d\xab\xc6\xf4hz\\\xbdl\xbd6\xbd\t}\x96\xbe\xa7~\x81~\xb7\xfe[\x035\x830\x83\xfd\x06\x03\x06\xdf\x0cM\r\x93\r\x9b\x0c\x9f\x19I\x18\xb9\x1b\x15\x18\xf5\x1a\xfdi\xacc\